# HELICoiD Evaluation

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt
%matplotlib widget

In [ ]:
import os
import sys
from pathlib import Path

src_dir = str(Path(os.getcwd()).parent / "src")
sys.path.insert(1, src_dir)
#os.environ["PYTHONPATH"] = src_dir

In [ ]:
import data_loader
import calculate_concentr_diffs
from concentr_diffs_pathlength import *
from plotting import *
from mbll_functions import *
from utils import *
from mcs_function import *
import config
import pickle
from ray import tune
import matplotlib.pyplot as plt
from inspect import signature
from tqdm.notebook import tqdm
import time
import pandas

## Setup

load the m-parameters for the Jaques models

In [ ]:
tissue_types_short = ["gray matter", "tumor", "blood vessel"]
tissue_types_long = ["general"] + tissue_types_short

In [ ]:
with open(config.m_params_path, "rb") as f:
    jacques_m_params, _, _, _ = pickle.load(f)

create MCS-Attenuation Object

In [ ]:
A_mcs = SimulationAttenuation(config.mcs_func_path)

In [ ]:
mcs_sparsity = 18 # results in 30 wavelengths

Load  data

In [ ]:
loader = data_loader.DataLoaderHELICOID(
    config.dataset_dir,
    wavelength_left_cut=520,
    wavelength_right_cut=900
)

mu_a_matrix = loader.absorption_coefs(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

delta_A_gt_img = loader.get_attenuation_change("020-01")
A_gt_img = loader.get_attenuation("020-01")

foreground_mask = (loader.label_map != 4)
labeled_mask = (loader.label_map != 0)
labeled_foreground_mask = labeled_mask & foreground_mask

In [ ]:
loader.wavelengths[::mcs_sparsity]

In [ ]:
len(loader.wavelengths[::mcs_sparsity])

In [ ]:
gray_matter_mcs_pathlength_data = np.loadtxt(config.pl_dir / "gray_matter_mcs.txt")
blood_vessel_mcs_pathlength_data = np.loadtxt(config.pl_dir / "blood_vessel_pure_mcs.txt")
tumor_mcs_pathlength_data = np.loadtxt(config.pl_dir / "tumor_mcs.txt")

In [ ]:
lengths = tuple()
for mcs_data in (gray_matter_mcs_pathlength_data, tumor_mcs_pathlength_data, blood_vessel_mcs_pathlength_data):
    pathlength = np.interp(loader.wavelengths, mcs_data[:, 0], mcs_data[:, 4] / 10) # convert to cm
    # notice the minus: mean scatterlength = mean pathlength - ((mean scattercount)/mu_s)
    scatterlength = mcs_data[:, 4] / 10 - mcs_data[:, 5] / (mcs_data[:, 2] * 10)
    scatterlength = np.interp(loader.wavelengths, mcs_data[:, 0], scatterlength)
    lengths += (pathlength, scatterlength)

gray_matter_pathlength, gray_matter_scatterlength, tumor_pathlength, tumor_scatterlength, blood_vessel_pathlength, blood_vessel_scatterlength = lengths

In [ ]:
gray_matter_baseline_attenuation = np.interp(
    loader.wavelengths,
    gray_matter_mcs_pathlength_data[:, 0],
    -np.log(gray_matter_mcs_pathlength_data[:, 3])
)

blood_vessel_baseline_attenuation = np.interp(
    loader.wavelengths,
    blood_vessel_mcs_pathlength_data[:, 0],
    -np.log(blood_vessel_mcs_pathlength_data[:, 3])
)

tumor_baseline_attenuation = np.interp(
    loader.wavelengths,
    tumor_mcs_pathlength_data[:, 0],
    -np.log(tumor_mcs_pathlength_data[:, 3])
)

Create NLLS + MCS objects that will be used several times

In [ ]:
params = {
    "wavelengths": loader.wavelengths,
    "mu_a_matrix": mu_a_matrix,
    "tissue_types": ["gray matter", "tumor", "blood vessel"],
    "pathlength": [gray_matter_pathlength, tumor_pathlength, blood_vessel_pathlength],
    "scatterlength": [gray_matter_scatterlength, tumor_scatterlength, blood_vessel_scatterlength],
    "baseline_b": [ data_loader.DataLoader.tissue_parameters[tissue][1][1] for tissue in tissue_types_short],
    "g": [data_loader.DataLoader.tissue_parameters[tissue][-1] for tissue in tissue_types_short],
}

A_mbll_mcs = A_mbll_fit(
    baseline_attenuation=[
        gray_matter_baseline_attenuation,
        tumor_baseline_attenuation,
        blood_vessel_baseline_attenuation
    ],
    baseline_concentrations = [
        data_loader.DataLoader.tissue_parameters[tissue][0] for tissue in tissue_types_short
    ],
    baseline_a = [
        data_loader.DataLoader.tissue_parameters[tissue][1][0] for tissue in tissue_types_short
    ],
    **params
)

A_mbll_mcs_delta = A_mbll_fit_delta(**params)

Load reference parameters obtained through hyperparameter search

In [ ]:
with open(config.reference_params_path, "rb") as f:
    hyperparams = pickle.load(f)

In [ ]:
config.reference_params_path

## Backward Test - HELICoiD

Plot HeliCoiD spectum of gray matter pixel, and pixel obtained by MCS + NLLS forward model for gray matter literature values.

In [ ]:
A_mcs_predicted = A_mcs.A_concentrations(
    loader.wavelengths,
    mu_a_matrix,
    data_loader.DataLoader.tissue_parameters["gray matter"][0],
    *data_loader.DataLoader.tissue_parameters["gray matter"][1]
)

In [ ]:
plot_spectra([A_gt_img[:, loader.reference_pixel[0], loader.reference_pixel[1]], A_mcs_predicted + 0.4] , loader.wavelengths)

### Absolute vs Difference

Create mask of all labeled pixels, without background pixels

In [ ]:
# every labeled pixel without background
print(np.count_nonzero(labeled_foreground_mask))

In [ ]:
# (models, parameters, spectrum)
concentrations_absolute_diff = np.zeros((6, 6, np.count_nonzero(labeled_foreground_mask)))

#### Jacques

Optimization using absolute spectra

In [ ]:
res, errors = concentr_fit_nonlinear_multiple_tissues_concurrent(
    A_gt_img[:, labeled_foreground_mask],
    loader.label_map[labeled_foreground_mask], # only contains 1, 2, 3 (normal, tumor, blood)
    loader.wavelengths,
    mu_a_matrix,
    A_jacques_blood_fraction,
    const_vals=[
        jacques_m_params["gray matter"], #normal
        jacques_m_params["tumor"], #tumor
        jacques_m_params["blood vessel"]  #blood
    ],
    variables_bool_arr=np.array([True] * 8 + [False] * 3),
    left_bounds = np.array([0] * 8),
    right_bounds = np.array([1, 1, 0.05, 0.05, 1, 1, 100, 7]),
    update_init=True,
    constraint=None,
    init_vals=concentrations_to_blood_fraction(
        np.concatenate(data_loader.DataLoader.tissue_parameters["gray matter"][:2])
    ),
    num_processes=20,
    progress_bar=True
)

concentrations_absolute_diff[0, :, :] = blood_fraction_to_concentrations(res[:6, :])

Optimize using difference spectrum

In [ ]:
res, errors = concentr_fit_nonlinear_multiple_tissues_concurrent(
    delta_A_gt_img[:, labeled_foreground_mask],
    loader.label_map[labeled_foreground_mask], # only contains 1, 2, 3 (unlabeled, normal, tumor, blood)
    loader.wavelengths,
    mu_a_matrix,
    A_jacques_blood_fraction,
    const_vals=[
        jacques_m_params["gray matter"], #normal
        jacques_m_params["tumor"], #tumor
        jacques_m_params["blood vessel"]  #blood
    ],
    ref_vals=hyperparams["020-01", "Jacques, specific"][1],
    variables_bool_arr=([True] * 8 + [False] * 3),
    left_bounds = np.array([0] * 8),
    right_bounds = np.array([1, 1, 0.05, 0.05, 1, 1, 100, 7]),
    update_init=True,
    constraint=None,
    init_vals=concentrations_to_blood_fraction(
        np.concatenate(data_loader.DataLoader.tissue_parameters["gray matter"][:2])
    ),
    num_processes=20,
    progress_bar=True
)

concentrations_absolute_diff[1, :, :] = blood_fraction_to_concentrations(res[:6, :])

#### NLLS + MCS (Skipped, due to very long runtime)

In [ ]:
res, errors = concentr_fit_nonlinear_multiple_tissues_concurrent(
    A_gt_img[::mcs_sparsity, labeled_foreground_mask],
    loader.label_map[labeled_foreground_mask], # only contains 0, 1, 2, 3 (unlabeled, normal, tumor, blood)
    loader.wavelengths[::mcs_sparsity],
    mu_a_matrix[::mcs_sparsity, :],
    A_mcs.A_blood_fraction,
    jacobian=A_mcs.jacobian_blood_fraction,
    variables_bool_arr=([True] * 8),
    left_bounds = np.array([0] * 8),
    right_bounds = np.array([1, 1, 0.05, 0.05, 1, 1, 100, 7]),
    update_init=True,
    constraint=None,
    init_vals=concentrations_to_blood_fraction(
        np.concatenate(data_loader.DataLoader.tissue_parameters["gray matter"][:2])
    ),
    num_processes=30,
    progress_bar=True
)

concentrations_absolute_diff[2, :, :] = blood_fraction_to_concentrations(res[:6, :])

Optimize using difference spectrum

In [ ]:
res, errors = concentr_fit_nonlinear_multiple_tissues_concurrent(
    delta_A_gt_img[::mcs_sparsity, labeled_foreground_mask],
    loader.label_map[labeled_foreground_mask], # only contains 0, 1, 2, 3 (unlabeled, normal, tumor, blood)
    loader.wavelengths[::mcs_sparsity],
    mu_a_matrix[::mcs_sparsity, :],
    A_mcs.A_blood_fraction,
    jacobian=A_mcs.jacobian_blood_fraction,
    variables_bool_arr=([True] * 8),
    ref_vals=hyperparams["020-01", "NLLS + MCS"][1],
    left_bounds = np.array([0] * 8),
    right_bounds = np.array([1, 1, 0.05, 0.05, 1, 1, 100, 7]),
    update_init=True,
    constraint=None,
    init_vals=concentrations_to_blood_fraction(
        np.concatenate(data_loader.DataLoader.tissue_parameters["gray matter"][:2])
    ),
    num_processes=30,
    progress_bar=True
)

concentrations_absolute_diff[3, :, :] = blood_fraction_to_concentrations(res[:6, :])

#### MBLL + MCS

In [ ]:
concentrations_absolute_diff[4, :, (loader.label_map[labeled_foreground_mask] == 1)] = A_mbll_mcs.concentr_fit(
    A_gt_img[:, (loader.label_map == 1)],
    "gray matter"
)[:6, :].T

concentrations_absolute_diff[4, :, (loader.label_map[labeled_foreground_mask] == 2)] = A_mbll_mcs.concentr_fit(
    A_gt_img[:, (loader.label_map == 2)],
    "tumor"
)[:6, :].T

concentrations_absolute_diff[4, :, (loader.label_map[labeled_foreground_mask] == 3)] = A_mbll_mcs.concentr_fit(
    A_gt_img[:, (loader.label_map == 3)],
    "blood vessel"
)[:6, :].T

# we compute concentration changes for every pixel as reference, and average results
#for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
#    tissue_pixels = list(zip(*np.nonzero(loader.label_map == tissue_idx)))
#    for ref_pixel_idx, ref_pixel_coords in tqdm(list(enumerate(tissue_pixels))):
#        delta_A_gt_img_tmp = A_gt_img[:, (loader.label_map == tissue_idx)] - A_gt_img[:, *ref_pixel_coords, None]
#        concentrations_absolute_diff[5, :6, (loader.label_map[labeled_foreground_mask] == tissue_idx)] += A_mbll_mcs_delta.concentr_fit_delta(
#            delta_A_gt_img_tmp,
#            tissue
#        )[:6, :].T
#        
#    concentrations_absolute_diff[5, :, (loader.label_map[labeled_foreground_mask] == tissue_idx)] /= len(tissue_pixels)

# simply use a single reference pixel

for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
    concentrations_absolute_diff[5, :6, (loader.label_map[labeled_foreground_mask] == tissue_idx)] = A_mbll_mcs_delta.concentr_fit_delta(
        delta_A_gt_img[:, (loader.label_map == tissue_idx)],
        tissue
    )


In [ ]:
# (tissue, models, parameters), models incl. GT
concentrations_absolute_diff_average = np.empty((3, 7, 6))

for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
    concentrations_absolute_diff_average[tissue_idx-1, 0, :] = data_loader.DataLoader.tissue_parameters[tissue][0]
    concentrations_absolute_diff_average[tissue_idx-1, 1:, :] = np.average(concentrations_absolute_diff[:, :, (loader.label_map[labeled_foreground_mask] == tissue_idx)], axis=-1)


In [ ]:
plot_concentrations_bar(concentrations_absolute_diff_average, ["Gray Matter", "Tumor", "Blood Vessel"], ["GT", "Abs. Jaq.", "Diff. Jaq.", "Abs. MCS", "Diff. MCS", "Abs. MBLL", "Diff. MBLL"], ["Hb02", "Hbb", "oxCCO", "redCCO", "Water", "Fat"])

In [ ]:
with open(config.eval_dir / "helicoid/concentrations_absolute_diff.pickle", "wb+") as f:
    pickle.dump(concentrations_absolute_diff, f)

with open(config.eval_dir / "helicoid/concentrations_absolute_diff_average.pickle", "wb+") as f:
    pickle.dump(concentrations_absolute_diff_average, f)

Save data.
For Diff MBLL simply use regular difference between two spectra. Also just use gray matter.

In [ ]:
bar_plot_data = concentrations_absolute_diff_average.copy()
bar_plot_data[:, ]

### Concentration Reconstruction

In [ ]:
fully_reconstructed_image_id = "020-01"
image_ids = ["020-01", "012-01", "015-01"]
num_images = len(image_ids)

Define additional MBLL backwards models

In [ ]:
baseline_attenuations_diffusion, baseline_pathlengths_diffusion, baseline_scatterlengths_diffusion = [], [], []
for tissue_type in tissue_types_short:
    params = (
        loader.wavelengths,
        mu_a_matrix,
        data_loader.DataLoader.tissue_parameters[tissue_type][0],
        *data_loader.DataLoader.tissue_parameters[tissue_type][1],
        *data_loader.DataLoader.tissue_parameters[tissue_type][-2:],
    )
    baseline_attenuations_diffusion.append(A_carp_concentrations(*params)[:, 0])
    baseline_pathlengths_diffusion.append(A_carp_pathlength(*params)[:, 0]),
    baseline_scatterlengths_diffusion.append(A_carp_scatterlength(*params)[:, 0])

A_mbll_carp = A_mbll_fit_delta(
    loader.wavelengths,
    mu_a_matrix,
    tissue_types_short,
    baseline_pathlengths_diffusion,
    baseline_scatterlengths_diffusion,
    [data_loader.DataLoader.tissue_parameters[tissue][1][1] for tissue in tissue_types_short],
    [data_loader.DataLoader.tissue_parameters[tissue][2] for tissue in tissue_types_short]
)

In [ ]:
A_mbll_mcs_gm_pl_const = A_mbll_fit_delta(
    loader.wavelengths,
    mu_a_matrix,
    ["general"],
    [gray_matter_baseline_attenuation],
    [gray_matter_scatterlength],
    [data_loader.DataLoader.tissue_parameters["gray matter"][1][1]],
    [data_loader.DataLoader.tissue_parameters["gray matter"][2]]
)

A_mbll_const = A_mbll_fit_delta(
    loader.wavelengths,
    mu_a_matrix,
    ["general"],
    np.ones((1, len(loader.wavelengths))),
    np.ones((1, len(loader.wavelengths))),
    [data_loader.DataLoader.tissue_parameters["gray matter"][1][1]],
    [data_loader.DataLoader.tissue_parameters["gray matter"][2]]
)

In [ ]:
backwards_models_mbll = {
    A_mbll_mcs_delta,
    A_mbll_carp
}

backwards_models_mbll_tissue_independent = {
    A_mbll_mcs_gm_pl_const,
    A_mbll_const
}

In [ ]:
backwards_models_nonlinear = {
    "Jacques, specific":
    (
        A_jacques_blood_fraction,
        {"const_vals": [list(jacques_m_params[tissue]) for tissue in tissue_types_long]}
    ),
    "Jacques, general":
    (
        A_jacques_blood_fraction, 
        {"const_vals": [list(jacques_m_params["general"])] * len(tissue_types_long)}
    ),
    "Carp":
    (
        A_carp_blood_fraction, 
        {
            "const_vals": [data_loader.DataLoader.tissue_parameters[tissue][-2:] for tissue in (["gray matter"] + tissue_types_short)],
        }
    ),
    "NLLS + MCS":
    (A_mcs.A_blood_fraction, {
        "jacobian": A_mcs.jacobian_blood_fraction
    })
}

In [ ]:
num_mbll_models = len(backwards_models_mbll)
num_mbll_models_tissue_independent = len(backwards_models_mbll_tissue_independent)
num_nonlinear_models = len(backwards_models_nonlinear)
loader.load_data(fully_reconstructed_image_id)
reconstructed_img_num_spectrum = np.prod(loader.get_attenuation(fully_reconstructed_image_id).shape[1:])

# (models, params, spectrum)
reconstructed_concentrations_full_image = np.ones((
    num_mbll_models + num_mbll_models_tissue_independent + num_nonlinear_models,
    8,
    reconstructed_img_num_spectrum
), dtype=float) * np.nan

# (models)
reconstruction_time = np.zeros(num_mbll_models + num_nonlinear_models + num_mbll_models_tissue_independent)

# (tissues, models, images, params)
#reconstructed_concentrations_average = np.empty((len(tissue_types), num_mbll_models + num_nonlinear_models, num_images, 8))


tissue_ctr = np.zeros((len(image_ids), 5), dtype=int)
for image_idx, image_id in enumerate(image_ids):
    loader.load_data(image_id)
    cur_label_map = loader.label_map
    labels, label_cts = np.unique(cur_label_map, return_counts=True)
    tissue_ctr[image_idx, labels.astype(int)] = label_cts

# tissue pixel counts summed over all images
tissue_ctr_summed = np.sum(tissue_ctr, axis=0, dtype=int)

# inhomogenous array
# first index indexes tissue
# after that the indices are (models, params, num_spectra)
# where num_spectra is the number of pixels of tissue type from all images of interest
reconstructed_concentrations_labeled = []
for tissue_idx in [1, 2, 3]:
    reconstructed_concentrations_labeled.append(
        np.empty((num_mbll_models + num_nonlinear_models, 8, tissue_ctr_summed[tissue_idx]), dtype=float)
    )


In [ ]:
tissue_ctr

MBLL - models, with tissue specific parameters

In [ ]:
for image_idx, image_id in enumerate(image_ids):
    A_gt_img = loader.get_attenuation(image_id)   
    for model_idx, mbll_model in tqdm(list(enumerate(backwards_models_mbll))):
        for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
            if image_id == fully_reconstructed_image_id and tissue == "gray matter":
                # additionally compute concentrations for unlabeled region,
                # use default gray matter pixel as reference pixel
                delta_A_gt_img = loader.get_attenuation_change(image_id, reference_label="gray matter")
                reconstruction_time[model_idx] = -time.monotonic()
                res = mbll_model.concentr_fit_delta(
                    delta_A=delta_A_gt_img[:, (loader.label_map == 0)],
                    tissue_type="gray matter"
                )
                reconstruction_time[model_idx] += time.monotonic()
                reconstructed_concentrations_full_image[model_idx, :7, np.flatnonzero(loader.label_map == 0)] = res.T
            
            reconstructed_concentrations_tmp = np.zeros((7, tissue_ctr[image_idx, tissue_idx], tissue_ctr[image_idx, tissue_idx]))
            tissue_pixels = list(zip(*np.nonzero(loader.label_map == tissue_idx)))
            time_val = -time.monotonic()
            for ref_idx, ref_coords in tqdm(list(enumerate(tissue_pixels))):
                delta_A_gt_img = A_gt_img[:, (loader.label_map == tissue_idx)] - A_gt_img[:, *ref_coords, None]
                res = mbll_model.concentr_fit_delta(
                    delta_A=delta_A_gt_img,
                    tissue_type=tissue
                )
                reconstructed_concentrations_tmp[:, ref_idx, :] = res
            time_val += time.monotonic()
            if image_id == fully_reconstructed_image_id:
                reconstruction_time[model_idx] += time_val / tissue_ctr[image_idx, tissue_idx]
            assert len(tissue_pixels) == tissue_ctr[image_idx, tissue_idx]
            cur_tissue_spectrum_idx = np.sum(tissue_ctr[:image_idx, tissue_idx])
            reconstructed_concentrations_labeled[tissue_idx-1][model_idx, :7, cur_tissue_spectrum_idx:(cur_tissue_spectrum_idx+tissue_ctr[image_idx, tissue_idx])] = np.average(reconstructed_concentrations_tmp, axis=1) #average over reference pixels
            if image_id == fully_reconstructed_image_id:
                # for image reconstruction, we only use one pixel within tissue as reference
                reconstructed_concentrations_full_image[model_idx, :7, np.flatnonzero(loader.label_map == tissue_idx)] = reconstructed_concentrations_tmp[:, loader.reference_pixel_tissue_ctr, :].T  

Old version of cell above. Should do the same, but for fewer pixels.

In [ ]:
num_ref_pixels = 100

In [ ]:
for image_idx, image_id in enumerate(image_ids):
    A_gt_img = loader.get_attenuation(image_id)    
    for model_idx, mbll_model in tqdm(list(enumerate(backwards_models_mbll))):
        for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
            if image_id == fully_reconstructed_image_id and tissue == "gray matter":
                # additionally compute concentrations for unlabeled region,
                # use default gray matter pixel as reference pixel
                delta_A_gt_img = loader.get_attenuation_change(image_id, reference_label="normal")
                res = mbll_model.concentr_fit_delta(
                    delta_A=delta_A_gt_img[:, (loader.label_map == 0)],
                    tissue_type=tissue
                )
                reconstructed_concentrations_full_image[model_idx, :7, (loader.label_map.reshape(-1) == 0)] = res.T
            
            # reconstruct parameters for all pixels of tissue, using all pixels of same tissue type as reference
            num_pixels_per_tissue = tissue_ctr[image_idx, tissue_idx]
            #(params, ref-spectrum, spectrum)
            reconstructed_concentrations_tmp = np.zeros((7, num_ref_pixels, num_pixels_per_tissue))
            ref_pixels = np.zeros_like(loader.label_map, dtype=bool)
            ref_pixels[np.unravel_index(np.flatnonzero(loader.label_map == tissue_idx)[:num_ref_pixels], shape=loader.label_map.shape)] = True
            delta_A_gt_img = A_gt_img[:, None, (loader.label_map == tissue_idx)] - A_gt_img[:, ref_pixels, None]
            res = mbll_model.concentr_fit_delta(
                delta_A=delta_A_gt_img,
                tissue_type=tissue
            )
            if image_id == fully_reconstructed_image_id:
                # for image reconstruction, we only use one pixel within tissue as reference
                reconstructed_concentrations_full_image[model_idx, :7, np.flatnonzero(loader.label_map == tissue_idx)] = res[:, loader.reference_pixel_tissue_ctr, :].T
            
            reconstructed_concentrations_labeled[tissue_idx-1][model_idx, :7, :] = np.average(res, axis=1)
            

Nonlinear models

In [ ]:
for image_idx, image_id in enumerate(image_ids):
    delta_A_gt_img = loader.get_attenuation(image_id)    
    labeled_foreground_mask = (loader.label_map != 0) & (loader.label_map != 4)
    for model_idx, model_str in tqdm(list(enumerate(backwards_models_nonlinear.keys(), start=len(backwards_models_mbll)))):
        delta_A_gt_img_tmp, wavelengths_tmp, mu_a_matrix_tmp = delta_A_gt_img, loader.wavelengths, mu_a_matrix
        model_func = backwards_models_nonlinear[model_str][0]
        
        if "NLLS" in model_str and "MCS" in model_str:
            delta_A_gt_img_tmp, wavelengths_tmp, mu_a_matrix_tmp = delta_A_gt_img[::mcs_sparsity, :], loader.wavelengths[::mcs_sparsity], mu_a_matrix[::mcs_sparsity, :]

        if image_id == fully_reconstructed_image_id:
                reconstruction_time[model_idx] = -time.monotonic()
                res, errors = concentr_fit_nonlinear_multiple_tissues_concurrent(
                    delta_A_gt_img_tmp[:, loader.label_map != 4],
                    loader.label_map[loader.label_map != 4],
                    wavelengths_tmp,
                    mu_a_matrix_tmp,
                    model_func,
                    variables_bool_arr=[True] * 8 + [False] * (len(signature(model_func).parameters) - 5),
                    left_bounds = np.array([0] * 8),
                    right_bounds = np.array([1, 1, 0.05, 0.05, 1, 1, 100, 7]),
                    update_init=True,
                    constraint=None,
                    init_vals=concentrations_to_blood_fraction(
                        np.concatenate(data_loader.DataLoader.tissue_parameters["gray matter"][:2])
                    ),
                    ref_vals=hyperparams[image_id, model_str][1],
                    num_processes=30,
                    **backwards_models_nonlinear[model_str][1]
                )
                reconstruction_time[model_idx] += time.monotonic()
                res = blood_fraction_to_concentrations(res)
                reconstructed_concentrations_full_image[model_idx, :, np.flatnonzero(loader.label_map != 4)] = res.T
                for tissue_idx in [1, 2, 3]:
                    cur_tissue_idx = np.sum(tissue_ctr[:image_idx, tissue_idx])
                    reconstructed_concentrations_labeled[tissue_idx-1][model_idx, :, cur_tissue_idx:(cur_tissue_idx+tissue_ctr[image_idx, tissue_idx])] = res[:, (loader.label_map[loader.label_map != 4] == tissue_idx)]
        else:
            res, errors = concentr_fit_nonlinear_multiple_tissues_concurrent(
                delta_A_gt_img_tmp[:, labeled_foreground_mask],
                loader.label_map[labeled_foreground_mask],
                wavelengths_tmp,
                mu_a_matrix_tmp,
                model_func,
                variables_bool_arr=([True] * 8) + ([False] * (len(signature(model_func).parameters) - 5)),
                left_bounds=np.array([0]*8),
                right_bounds=np.array([1, 1, 0.05, 0.05, 1, 1, 100, 7]),
                update_init=True,
                constraint=None,
                init_vals=concentrations_to_blood_fraction(
                    np.concatenate(data_loader.DataLoader.tissue_parameters["gray matter"][:2])
                ),
                ref_vals=hyperparams[image_id, model_str][1],
                num_processes=30,
                **backwards_models_nonlinear[model_str][1]
            )
            res = blood_fraction_to_concentrations(res)
            for tissue_idx in [1, 2, 3]:
                cur_tissue_idx = np.sum(tissue_ctr[:image_idx, tissue_idx])
                reconstructed_concentrations_labeled[tissue_idx-1][model_idx, :, cur_tissue_idx:(cur_tissue_idx+tissue_ctr[image_idx,tissue_idx])] = res[:, (loader.label_map[labeled_foreground_mask] == tissue_idx)]


Tissue independent MBLL models
(only image reconstruction)

In [ ]:
delta_A_gt_img = loader.get_attenuation_change(fully_reconstructed_image_id)
for model_idx, model in tqdm(list(enumerate(backwards_models_mbll_tissue_independent, start=(num_mbll_models + num_nonlinear_models)))):
    res = model.concentr_fit_delta(
        delta_A=delta_A_gt_img[:, loader.label_map != 4],
        tissue_type="general"
    )

    reconstructed_concentrations_full_image[model_idx, :7, np.flatnonzero(loader.label_map != 4)] = res.T

In [ ]:
with open(config.eval_dir/"helicoid/reconstructed_concentrations_full_image.pickle", "wb+") as f:
    pickle.dump(reconstructed_concentrations_full_image, f)

In [ ]:
with ope

Compute statistics

First data for bar plot: Per Tissue average over all spectra

In [ ]:
# (tissues, models, params)
bar_plot_data = np.zeros((3, num_mbll_models + num_nonlinear_models, 6), dtype=float)

for tissue_idx in [1, 2, 3]:
    bar_plot_data[tissue_idx-1] = np.average(reconstructed_concentrations_labeled[tissue_idx-1][:, :6, :], axis=-1)

Now the prediction error for the appendix.
For nonlinear models, for every tissue we take the data from the bar plots compute squared error of (averaged, in previous step) parameters to literature values. For the tissue average, average these, weighted by the number of pixels per tissue type. For MBLL models the tissue weights have to be squared, since we computed average over all reference pixels.

For MBLL we do the same, but don't substract any literature values.

In [ ]:
# (models, tissue+1)
prediction_error_table = np.empty(
    (
        num_nonlinear_models + num_mbll_models,
        4 # three tissue types + one column for average over all tissues
    )
)

for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
    prediction_error_table[:num_mbll_models, tissue_idx-1] = np.sqrt(np.sum(bar_plot_data[tissue_idx-1, :num_mbll_models, :6]**2, axis=-1))
    prediction_error_table[num_mbll_models:, tissue_idx-1] = np.sqrt(np.sum((bar_plot_data[tissue_idx-1, num_mbll_models:, :6] - 
        data_loader.DataLoader.tissue_parameters[tissue][0])**2, axis=-1))



prediction_error_table[:num_mbll_models, 3] = np.average(
    prediction_error_table[:num_mbll_models, :3],
    axis=1,
    weights=tissue_ctr_summed[1:4]**2
)

prediction_error_table[num_mbll_models:, 3] = np.average(
    prediction_error_table[num_mbll_models:, :3],
    axis=1,
    weights=tissue_ctr_summed[1:4]
)

In [ ]:
df = pandas.DataFrame(prediction_error_table)
df.index = ["MBLL MCS", "MBLL Carp"] + list(backwards_models_nonlinear.keys())
df.columns = ["GM", "tumor", "blood vessel", "avg"]
print(df)

Store the data.

In [ ]:
with open(config.eval_dir/"helicoid/prediction_error_df.pickle", "wb+") as f:
    pickle.dump(df, f)

### Diagrams

#### Images

To visualize MBLL images, literature values are added to concentration-change values.

In [ ]:
image_data = reconstructed_concentrations_full_image.copy()
for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
    image_data[:num_mbll_models, :7, :] += np.concatenate(data_loader.DataLoader.tissue_parameters[tissue][:2])[None, :7, None]

loader.load_data(fully_reconstructed_image_id)
image_data = image_data[:, :6, :].reshape(image_data.shape[0], 6, *loader.label_map.shape)

#### Bar plot

append literature values

In [ ]:
literature_bar_data = np.empty((bar_plot_data.shape[0], 1, 6))
for tissue_idx, tissue in enumerate(tissue_types_short, start=1):
    literature_bar_data[tissue_idx-1:, 0, :] = data_loader.DataLoader.tissue_parameters[tissue][0]

bar_plot_data = np.concatenate((literature_bar_data, bar_plot_data), axis=1)

In [ ]:
with open(config.eval_dir/"helicoid/bar_plot_data.pickle", "wb+") as f:
    pickle.dump(bar_plot_data, f)

In [ ]:
plot_concentrations_bar(
    bar_plot_data,
    tissue_types_short,
    ["GT"] + ["MBLL1", "MBLL2"] + list(backwards_models_nonlinear.keys()),
    ["Hb02", "Hbb", "oxCCO", "redCCO", "f_H20", "f_fat"]
)

Reconstruction times

In [ ]:
df = pandas.DataFrame(reconstruction_time.T)
df.index = ["MBLL MCS", "MBLL Carp", "MBLL, const. GM PL", "MBLL ones PL"] + list(backwards_models_nonlinear.keys())
df.columns = ["Runtime [s]"]
print(df)

In [ ]:
with open(config.eval_dir/"helicoid/times_df.pickle", "wb+") as f:
    pickle.dump(df, f)